# Web Scraper for finviz 

<img src="finviz6.png" width="200"/>

In [1]:
from time import sleep
from urllib.request import Request, urlopen

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [ ]:
# Main Page -- https://finviz.com/screener.ashx?v=152
# The columns must be selected before the web scraping starts

In [ ]:
# r=1 for the first page (results will appears from 20 to 20 (r=21, r=41, etc))
# Columns must be selected in advance and use the complete URL (c=xi,...,xn)

url = "https://finviz.com/screener.ashx?v=152&r=21&c=0,1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19,29,39,43,51,53,54,65,66,67"

In [ ]:
# Need to user urllib.request. Finviz does not allow request from python

req = Request(url, headers={"User-Agent": "Mozilla/5.0"})
webpage = urlopen(req).read()
html = BeautifulSoup(webpage, "html.parser")

In [ ]:
# Let's find the number of pages we have to go through (Improvement potential)

pages_num = html.findAll("option", {"value": "1"})
for i in pages_num:
    a = i.text
    a = a.split()
    a = a[1]
    a = a.replace("/", " ")
    a = a.split()
    pages = a[1]

In [ ]:
# Let's find the columns we have and store them in a list

columns = html.findAll("td", {"class": "table-top"})
list_columns = [column.text for column in columns]

In [ ]:
# Let's look through all the stocks

data_raw = []
list_of_stocks = html.findAll("tr", {"valign": "top"})
list_of_stocks
for stock in list_of_stocks:
    list_data_raw = stock.findAll("a", {"class": "screener-link"})
    list_data = [info.text for info in list_data_raw]
    data_raw.append(list_data)
data_raw

## Now let's put everything together in a function and add some features

In [11]:
def scrap_finviz(url, urlpages):

    #     Need to user urllib.request. Finviz does not allow request from python

    req = Request(url, headers={"User-Agent": "Mozilla/5.0"})
    webpage = urlopen(req).read()
    html = BeautifulSoup(webpage, "html.parser")

    pages_num = html.findAll("option", {"value": "1"})
    for i in pages_num:
        a = i.text
        a = a.split()
        a = a[1]
        a = a.replace("/", " ")
        a = a.split()
        pages = a[1]
        pages = int(a[1]) * 20

    #     Let's find the columns for our data frame (multiple classes form some columns)

    columns = html.findAll("td", {"class": ["table-top", "table-top-s"]})
    list_columns = [column.text for column in columns]
    list_columns

    #     Let's look through all the stocks and all the pages and store them in a list (tqdm added // multiple classes for the 2 types of rows and stocks )

    data_raw = []
    for i in tqdm(range(pages - pages + 1, pages, 20)):
        req1 = Request(urlpages(i), headers={"User-Agent": "Mozilla/5.0"})
        weburlpages = urlopen(req1).read()
        soup = BeautifulSoup(weburlpages, "html.parser")

        list_of_stocks = soup.findAll(
            "tr",
            {"valign": "top", "class": [
                "table-dark-row-cp", "table-light-row-cp"]},
        )

        for stock in list_of_stocks:
            list_data_raw = stock.findAll(
                "a", {"class": ["screener-link-primary", "screener-link"]}
            )
            list_data = [info.text for info in list_data_raw]
            data_raw.append(list_data)

    #        Adding the data to a data frame

    df = pd.DataFrame(data_raw, columns=list_columns)

    return list_columns, data_raw

In [3]:
# Some work on the URL added to simplify the process

url = input(
    'Please enter the url from finviz (use the complete link with c=x,...,xn for the selected columns ): \n\t For example: "https://finviz.com/screener.ashx?v=152&s=ta_topgainers&c=1,2,3,4,5,6" \n'
)
url1 = url.replace("c=", "r=1&c=")
urlpages = url.replace("c=", "r={i}&c=")
def urlpages1(i): return f"{urlpages}"


x, y = scrap_finviz(url1, urlpages1)

Please enter the url from finviz (use the complete link with c=x,...,xn for the selected columns ): 
	 For example: "https://finviz.com/screener.ashx?v=152&s=ta_topgainers&c=1,2,3,4,5,6" 
 https://finviz.com/screener.ashx?v=152&c=0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70


In [2]:
df = pd.DataFrame(y, columns=x)

NameError: name 'y' is not defined

In [3]:
df

NameError: name 'df' is not defined

In [12]:
df.to_csv("df_stocks.csv")

In [5]:
df = pd.read_csv('df_stocks.csv')

In [12]:
df.dtypes

Unnamed: 0         int64
No.                int64
Ticker            object
Company           object
Sector            object
Industry          object
Country           object
Market Cap        object
P/E               object
Fwd P/E           object
PEG               object
P/S               object
P/B               object
P/C               object
P/FCF             object
Dividend          object
Payout Ratio      object
EPS               object
EPS this Y        object
EPS next Y        object
EPS past 5Y       object
EPS next 5Y       object
Sales past 5Y     object
EPS Q/Q           object
Sales Q/Q         object
Outstanding       object
Float             object
Insider Own       object
Insider Trans     object
Inst Own          object
Inst Trans        object
Float Short       object
Short Ratio       object
ROA               object
ROE               object
ROI               object
Curr R            object
Quick R           object
LTDebt/Eq         object
Debt/Eq           object


In [15]:
df

,Unnamed: 0,No.,Ticker,Company,Sector,Industry,Country,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Dividend,Payout Ratio,EPS,EPS this Y,EPS next Y,EPS past 5Y,EPS next 5Y,Sales past 5Y,EPS Q/Q,Sales Q/Q,Outstanding,Float,Insider Own,Insider Trans,Inst Own,Inst Trans,Float Short,Short Ratio,ROA,ROE,ROI,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M,Oper M,Profit M,Perf Week,Perf Month,Perf Quart,Perf Half,Perf Year,Perf YTD,Beta,ATR,Volatility W,Volatility M,SMA20,SMA50,SMA200,50D High,50D Low,52W High,52W Low,RSI,from Open,Gap,Recom,Avg Volume,Rel Volume,Price,Change,Volume,Earnings,Target Price,IPO Date
0,0,1,A,"Agilent Technologies, Inc.",Healthcare,Diagnostics & Research,USA,33.51B,50.11,30.12,5.22,6.42,6.88,24.67,64.69,0.70%,30.80%,2.21,26.10%,14.03%,37.50%,9.60%,5.00%,5.50%,-1.00%,309.00M,307.28M,0.40%,-11.59%,89.80%,-1.02%,1.02%,2.53,7.30%,14.30%,15.20%,2.50,1.90,0.46,0.47,53.30%,15.30%,13.20%,0.65%,3.86%,13.62%,28.75%,41.62%,29.98%,1.00,2.85,2.89%,2.52%,3.02%,6.45%,23.32%,-5.63%,16.19%,-5.63%,81.40%,60.75,2.07%,-0.04%,2.20,1.24M,2.24,110.89,2.03%,"2,774,924",Nov 23/a,106.40,11/18/1999
1,1,2,AA,Alcoa Corporation,Basic Materials,Aluminum,USA,3.52B,-,272.75,-,0.38,1.03,2.03,14.54,-,-,-2.53,-559.70%,104.60%,-34.10%,-,-4.50%,77.80%,-7.90%,186.00M,184.84M,0.20%,0.32%,76.90%,2.30%,5.45%,1.92,-3.40%,-12.10%,-9.60%,1.80,1.20,0.75,0.75,13.80%,-0.20%,-5.00%,13.58%,37.77%,24.47%,114.84%,-6.60%,-12.51%,2.49,0.80,4.62%,4.70%,25.62%,39.00%,61.27%,-0.90%,71.40%,-14.14%,264.73%,80.19,-0.37%,-0.21%,2.60,5.24M,0.91,18.82,-0.58%,"4,748,037",Oct 14/a,14.43,10/18/2016
2,2,3,AAAU,Perth Mint Physical Gold ETF,Financial,Exchange Traded Fund,USA,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-0.80%,-1.74%,-2.86%,9.38%,27.65%,23.10%,-,0.20,0.63%,0.69%,-0.98%,-1.55%,4.54%,-4.99%,1.17%,-9.58%,28.71%,46.01,-0.21%,0.54%,-,636.92K,0.31,18.65,0.32%,"200,180",-,-,8/15/2018
3,3,4,AACG,ATA Creativity Global,Consumer Defensive,Education & Training Services,China,36.80M,-,-,-,1.57,1.01,2.16,-,-,-,-0.61,-37.70%,-,-48.00%,25.00%,-22.50%,154.90%,3.90%,34.43M,31.50M,18.70%,0.00%,0.10%,-11.11%,0.00%,0.00,-16.40%,-38.80%,-45.00%,0.50,0.50,0.00,0.01,34.50%,-,-68.30%,1.47%,-8.73%,6.48%,27.79%,-37.87%,-15.50%,1.63,0.07,4.61%,4.64%,1.19%,1.92%,8.71%,-19.58%,8.49%,-38.83%,87.91%,51.72,-2.54%,1.72%,-,74.61K,0.44,1.15,-0.86%,"32,622",Nov 12/a,-,1/29/2008
4,4,5,AACQ,Artius Acquisition Inc.,Financial,Shell Companies,USA,716.53M,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,72.45M,-,-,-,-,-,-,0.04,-,-,-,-,-,-,-,-,-,-,1.12%,1.12%,-,-,-,-1.10%,-,0.08,0.76%,0.81%,1.65%,0.76%,0.64%,-5.63%,3.29%,-5.63%,3.29%,63.74,0.20%,0.00%,-,327.83K,0.10,9.89,0.20%,"32,413",-,-,9/4/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7655,7655,16,AAT,"American Assets Trust, Inc.",Real Estate,REIT - Diversified,USA,1.74B,45.30,39.64,15.10,4.82,1.33,11.23,-,3.50%,166.80%,0.63,101.50%,38.46%,10.50%,3.00%,7.10%,-61.20%,-14.20%,59.83M,59.59M,0.30%,233.64%,97.20%,0.22%,1.14%,2.02,1.30%,2.90%,4.30%,-,-,1.10,1.10,65.10%,28.20%,10.40%,7.29%,17.26%,10.75%,5.70%,-38.10%,-37.82%,1.21,1.20,4.12%,4.55%,17.66%,16.40%,1.64%,-3.09%,37.67%,-40.73%,41.64%,64.61,-0.70%,-0.66%,2.80,336.05K,0.80,28.54,-1.35%,"268,419",Oct 27/a,27.00,1/13/2011
7656,7656,17,AAU,Almaden Minerals Ltd.,Basic Materials,Gold,Canada,131.90M,-,-,-,-,2.35,48.85,-,-,-,-0.02,-3.20%,-,31.70%,-,-,81.30%,-,119.16M,115.02M,7.20%,0.00%,5.90%,45.40%,0.49%,0.80,-4.20%,-4.60%,-,9.50,9.50,0.04,0.04,-,-,-,-2.70%,14.50%,59.57%,148.45%,129.79%,83.36%,1.19,0.10,7.16%,9.62%,1.50%,13.36%,79.53%,-12.90%,60.21%,-12.90%,409.19%,52.97,-3.57%,0.90%,-,696.81K,0.43,1.08,-2.70%,"298,754",-,-,10/17/2003
7657,7657,18,AAWW,"Atlas Air Worldwide Holdings, Inc.",Industrials,Airports & Air Services,USA,1.50B,-,6.18,-,0.50,0

In [14]:
pd.set_option('display.max_columns', 80)